In [99]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

In [100]:
# This path will be active after the launch of the hackathon
teamname = '9417-brhuang-unsw'
data_folder='s3://tf-trachack-data/212/'
# change root_folder to your team's root folder
# s3://tf-trachack-notebooks/<this should be replaced by team name as provided in EMAIL>/jupyter/jovyan/
root_folder='s3://tf-trachack-notebooks/'+'9417-brhuang-unsw'+'/jupyter/jovyan/'

df = pd.read_csv(root_folder+"sample-notebook/final_dataset.csv")  

In [101]:
df = df.drop(['date_observed','sim_size'], axis=1)
Y = df['upgrade']
X = df.drop(['upgrade','line_id'], axis=1)

In [104]:
##Initial testing random forest 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

rf = RandomForestClassifier(random_state=2)
rf.fit(X_train, Y_train)
y_pred = rf.predict(X_test)
acc = accuracy_score(Y_test, y_pred)
print(acc)
f1 = f1_score(Y_test, y_pred)
print(f1)

RandomForestClassifier(random_state=2)

In [111]:
##Read in cleaned test data 

df1 = pd.read_csv(root_folder+"sample-notebook/test_data/test_final_dataset.csv")  



feature_list = [col for col in df.columns if col not in df1.columns][1:]
zero_data = np.zeros(shape=(len(df1),len(feature_list)))
d = pd.DataFrame(zero_data, columns=feature_list)

df1 = pd.concat([df1, d], axis=1)

ids = df1['line_id']
df1 = df1.drop(['date_observed','sim_size', 'line_id'], axis=1)




In [113]:
##Train on best paramaters, and make final predictions
final_rf = RandomForestClassifier(n_estimators=1260, min_samples_split=6, min_samples_leaf=1, max_features='sqrt', max_depth=65, bootstrap=False, random_state=2)
final_rf.fit(X_train, Y_train)

X_final = df1
y_final_pred = final_rf.predict(X_final)


data = {
    'line_id': ids, 
    'upgrade': y_final_pred
}
final_predictions = pd.DataFrame(data)


final_predictions.to_csv(root_folder+"sample-notebook/final_predictions.csv", index=False)

RandomForestClassifier(bootstrap=False, max_depth=65, max_features='sqrt',
                       min_samples_split=6, n_estimators=1260, random_state=2)

In [37]:
##Do random grid search
param_grid = {'min_samples_leaf': np.arange(2, 50, 1)}
grid_tree = GridSearchCV(RandomForestClassifier(random_state=0),param_grid, cv=10, scoring='f1')
grid_tree.fit(X_train, Y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=0),
             param_grid={'min_samples_leaf': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             scoring='roc_auc')

In [61]:
print(grid_tree.best_params_['min_samples_leaf'])

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [62]:
param_grid = {'min_samples_split': np.arange(2, 40, 1)}

In [63]:
grid_tree = GridSearchCV(RandomForestClassifier(random_state=0),param_grid, cv=10, scoring='f1')
grid_tree.fit(X_train, Y_train)
print(grid_tree.best_params_['min_samples_split'])

2


In [123]:
from sklearn.model_selection import RandomizedSearchCV
##Code sourced from https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 344, 488, 633, 777, 922, 1066, 1211, 1355, 1500], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [124]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 12.6min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 58.9min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=2),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 344, 488, 633,
                                                         777, 922, 1066, 1211,
                                                         1355, 1500]},
                   random_state=42, verbose=2)

In [125]:
rf_random.best_params_


{'n_estimators': 1211,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 70,
 'bootstrap': False}

In [128]:
final_rf = RandomForestClassifier(n_estimators=1211, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', max_depth=70, bootstrap=False, random_state=2)
final_rf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=False, max_depth=70, max_features='sqrt',
                       min_samples_split=5, n_estimators=1211, random_state=2)

In [129]:
##Model with hyperparameter from random search 
y_pred = final_rf.predict(X_test)
acc = accuracy_score(Y_test, y_pred)
print(acc)
f1 = f1_score(Y_test, y_pred)
print(f1)

0.9182029711831037
0.8427932576539389


In [8]:
param_grid = {
    'bootstrap': [False],
    'max_depth': [65, 70, 75],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [100, 1200, 1211, 1260]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [9]:
grid_search.fit(X_train, Y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 18.4min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 61.7min finished


GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [65, 70, 75],
                         'max_features': ['sqrt'], 'min_samples_leaf': [1],
                         'min_samples_split': [4, 5, 6],
                         'n_estimators': [100, 1200, 1211, 1260]},
             verbose=2)

In [10]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 65,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 6,
 'n_estimators': 1260}

In [9]:
##Model with hyperparameter from random search 

final_rf = RandomForestClassifier(n_estimators=1260, min_samples_split=6, min_samples_leaf=1, max_features='sqrt', max_depth=65, bootstrap=False, random_state=3)
final_rf.fit(X_train, Y_train)
y_pred = final_rf.predict(X_test)
acc = accuracy_score(Y_test, y_pred)
print(acc)
f1 = f1_score(Y_test, y_pred)
print(f1)

0.9162341149096116
0.8396711202466598


In [3]:
param_grid = {
    'bootstrap': [False],
    'max_depth': [65, 70, 75],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1],
    'min_samples_split': [4, 5, 6],
    'n_estimators': [100, 1200, 1211, 1260]
}

In [4]:
print(param_grid)

{'bootstrap': [False], 'max_depth': [65, 70, 75], 'max_features': ['sqrt'], 'min_samples_leaf': [1], 'min_samples_split': [4, 5, 6], 'n_estimators': [100, 1200, 1211, 1260]}


In [23]:
final_rf.feature_importances_



array([2.20961066e-02, 2.22143664e-02, 1.33634512e-02, 4.13958691e-02,
       9.10256594e-02, 1.88128924e-02, 6.95048447e-03, 3.90864135e-03,
       2.38727454e-03, 9.51788111e-02, 2.61057168e-02, 5.68164714e-03,
       4.30874937e-03, 2.42724856e-04, 7.82842804e-02, 3.28017946e-02,
       3.29904669e-02, 4.86550550e-02, 3.48456830e-02, 6.82670647e-03,
       1.44017909e-02, 8.06693237e-03, 5.47206980e-03, 9.27581727e-03,
       1.44542567e-03, 2.33683713e-03, 2.55549964e-03, 2.64766114e-03,
       1.58778243e-03, 1.19901469e-03, 1.74127324e-03, 1.50345529e-03,
       1.90157613e-03, 1.87220354e-03, 1.72291796e-03, 1.42564559e-03,
       9.38351300e-04, 2.82228889e-03, 1.85401375e-03, 1.59829925e-03,
       7.05669749e-05, 1.93389066e-03, 2.47953943e-03, 1.01914375e-03,
       1.51498299e-03, 1.60948450e-02, 2.09563365e-03, 1.95493390e-03,
       1.39830011e-03, 1.47316631e-03, 1.79594641e-03, 2.55473782e-03,
       1.94490857e-03, 1.85518132e-03, 1.30985383e-03, 2.22722507e-03,
      

In [115]:
##Transform feature importance to dataframe
X.shape[1]

86

In [116]:
cols = X

In [117]:
cols = cols[0:0]

In [118]:
cols

,quantity,total_quantity,lrp_enrolled,length_of_membership,last_interaction,carrier_carrier 1,carrier_carrier 2,carrier_carrier 3,plan_name_Other,plan_name_not_given,plan_name_plan 1,plan_name_plan 2,plan_name_plan 3,plan_name_plan 4,total_redemptions,total_suspensions,total_unsuspensions,total_deactivations,total_reactivations,reactivation_channel_APP,reactivation_channel_IVR,reactivation_channel_SMS,reactivation_channel_TAS,reactivation_channel_WEB,reactivation_channel_other,cpu_cores_2,cpu_cores_2+4,cpu_cores_4,cpu_cores_8,cpu_cores_none,cpu_cores_other,expandable_storage_-1.0,expandable_storage_0.0,expandable_storage_1.0,gsma_device_type_Smartphone,gsma_device_type_none,gsma_device_type_other,gsma_operating_system_Android,gsma_operating_system_iOS,gsma_operating_system_none,gsma_operating_system_other,internal_storage_capacity_16,internal_storage_capacity_16/32/64/128,internal_storage_capacity_32/128/256,internal_storage_capacity_none,internal_storage_capacity_other,lte_category_-1.0,lte_category_4.0,lte_category_6.0,lte_category_9.0,lte_category_12.0,lte_category_16.0,lte_category_other,manufacturer_Apple,manufacturer_LG,manufacturer_Samsung,manufacturer_none,manufacturer_other,os_name_Android,os_name_iOS,os_name_none,os_name_other,os_version_10,os_version_11,os_version_12,os_version_6.0.1,os_version_7,os_version_8,os_version_8.1,os_version_9,os_version_none,os_version_other,total_ram_1024,total_ram_2048,total_ram_3072,total_ram_4096,total_ram_none,total_ram_other,hotspot_kb,total_kb,voice_count_total,voice_min_total,mms_total,sms_total,length_of_connection,used_network


In [119]:
cols.loc['y'] = final_rf.feature_importances_

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [120]:
cols

,quantity,total_quantity,lrp_enrolled,length_of_membership,last_interaction,carrier_carrier 1,carrier_carrier 2,carrier_carrier 3,plan_name_Other,plan_name_not_given,plan_name_plan 1,plan_name_plan 2,plan_name_plan 3,plan_name_plan 4,total_redemptions,total_suspensions,total_unsuspensions,total_deactivations,total_reactivations,reactivation_channel_APP,reactivation_channel_IVR,reactivation_channel_SMS,reactivation_channel_TAS,reactivation_channel_WEB,reactivation_channel_other,cpu_cores_2,cpu_cores_2+4,cpu_cores_4,cpu_cores_8,cpu_cores_none,cpu_cores_other,expandable_storage_-1.0,expandable_storage_0.0,expandable_storage_1.0,gsma_device_type_Smartphone,gsma_device_type_none,gsma_device_type_other,gsma_operating_system_Android,gsma_operating_system_iOS,gsma_operating_system_none,gsma_operating_system_other,internal_storage_capacity_16,internal_storage_capacity_16/32/64/128,internal_storage_capacity_32/128/256,internal_storage_capacity_none,internal_storage_capacity_other,lte_category_-1.0,lte_category_4.0,lte_category_6.0,lte_category_9.0,lte_category_12.0,lte_category_16.0,lte_category_other,manufacturer_Apple,manufacturer_LG,manufacturer_Samsung,manufacturer_none,manufacturer_other,os_name_Android,os_name_iOS,os_name_none,os_name_other,os_version_10,os_version_11,os_version_12,os_version_6.0.1,os_version_7,os_version_8,os_version_8.1,os_version_9,os_version_none,os_version_other,total_ram_1024,total_ram_2048,total_ram_3072,total_ram_4096,total_ram_none,total_ram_other,hotspot_kb,total_kb,voice_count_total,voice_min_total,mms_total,sms_total,length_of_connection,used_network
y,0.022312,0.022191,0.01404,0.041964,0.092887,0.018296,0.007125,0.003747,0.002589,0.096165,0.025858,0.00553,0.004095,0.000277,0.079172,0.032775,0.032234,0.046519,0.035054,0.00689,0.014482,0.008367,0.005373,0.009334,0.001433,0.002431,0.002801,0.002565,0.001913,0.001216,0.001629,0.001516,0.001896,0.001897,0.001798,0.001406,0.001037,0.002887,0.001905,0.001589,0.000085,0.001998,0.002424,0.001107,0.001566,0.016303,0.002073,0.002025,0.001356,0.001497,0.001829,0.002285,0.002079,0.001793,0.001365,0.002666,0.001392,0.001371,0.003619,0.001728,0.001402,0.000469,0.000982,0.000958,0.00085,0.001852,0.001276,0.001461,0.001231,0.001773,0.001474,0.002718,0.001137,0.005298,0.002299,0.005998,0.001282,0.001905,0.010692,0.032714,0.034336,0.041234,0.035665,0.04976,0.051508,0.001972


In [121]:
pd.set_option('display.max_columns', None)

In [122]:
by = list(cols)


In [123]:
cols.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort')

,quantity,total_quantity,lrp_enrolled,length_of_membership,last_interaction,carrier_carrier 1,carrier_carrier 2,carrier_carrier 3,plan_name_Other,plan_name_not_given,plan_name_plan 1,plan_name_plan 2,plan_name_plan 3,plan_name_plan 4,total_redemptions,total_suspensions,total_unsuspensions,total_deactivations,total_reactivations,reactivation_channel_APP,reactivation_channel_IVR,reactivation_channel_SMS,reactivation_channel_TAS,reactivation_channel_WEB,reactivation_channel_other,cpu_cores_2,cpu_cores_2+4,cpu_cores_4,cpu_cores_8,cpu_cores_none,cpu_cores_other,expandable_storage_-1.0,expandable_storage_0.0,expandable_storage_1.0,gsma_device_type_Smartphone,gsma_device_type_none,gsma_device_type_other,gsma_operating_system_Android,gsma_operating_system_iOS,gsma_operating_system_none,gsma_operating_system_other,internal_storage_capacity_16,internal_storage_capacity_16/32/64/128,internal_storage_capacity_32/128/256,internal_storage_capacity_none,internal_storage_capacity_other,lte_category_-1.0,lte_category_4.0,lte_category_6.0,lte_category_9.0,lte_category_12.0,lte_category_16.0,lte_category_other,manufacturer_Apple,manufacturer_LG,manufacturer_Samsung,manufacturer_none,manufacturer_other,os_name_Android,os_name_iOS,os_name_none,os_name_other,os_version_10,os_version_11,os_version_12,os_version_6.0.1,os_version_7,os_version_8,os_version_8.1,os_version_9,os_version_none,os_version_other,total_ram_1024,total_ram_2048,total_ram_3072,total_ram_4096,total_ram_none,total_ram_other,hotspot_kb,total_kb,voice_count_total,voice_min_total,mms_total,sms_total,length_of_connection,used_network
y,0.022312,0.022191,0.01404,0.041964,0.092887,0.018296,0.007125,0.003747,0.002589,0.096165,0.025858,0.00553,0.004095,0.000277,0.079172,0.032775,0.032234,0.046519,0.035054,0.00689,0.014482,0.008367,0.005373,0.009334,0.001433,0.002431,0.002801,0.002565,0.001913,0.001216,0.001629,0.001516,0.001896,0.001897,0.001798,0.001406,0.001037,0.002887,0.001905,0.001589,0.000085,0.001998,0.002424,0.001107,0.001566,0.016303,0.002073,0.002025,0.001356,0.001497,0.001829,0.002285,0.002079,0.001793,0.001365,0.002666,0.001392,0.001371,0.003619,0.001728,0.001402,0.000469,0.000982,0.000958,0.00085,0.001852,0.001276,0.001461,0.001231,0.001773,0.001474,0.002718,0.001137,0.005298,0.002299,0.005998,0.001282,0.001905,0.010692,0.032714,0.034336,0.041234,0.035665,0.04976,0.051508,0.001972


In [124]:
import numpy as np

In [125]:
cols = X
cols = cols[0:0]
#cols.reshape[]

In [126]:
a = final_rf.feature_importances_.reshape(1,86)


In [127]:
d = pd.DataFrame() 

In [128]:
d['value'] = final_rf.feature_importances_

In [129]:
a.shape

(1, 86)

In [130]:
d['feature'] = list(cols)

In [131]:
d.sort_values(by=['value'], ascending=False)

,value,feature
9,0.096165,plan_name_not_given
4,0.092887,last_interaction
14,0.079172,total_redemptions
84,0.051508,length_of_connection
83,0.049760,sms_total
17,0.046519,total_deactivations
3,0.041964,length_of_membership
81,0.041234,voice_min_total
82,0.035665,mms_total
18,0.035054,total_reactivations


In [132]:
d = d[['feature', 'value']]

In [133]:
d

,feature,value
0,quantity,0.022312
1,total_quantity,0.022191
2,lrp_enrolled,0.014040
3,length_of_membership,0.041964
4,last_interaction,0.092887
5,carrier_carrier 1,0.018296
6,carrier_carrier 2,0.007125
7,carrier_carrier 3,0.003747
8,plan_name_Other,0.002589
9,plan_name_not_given,0.096165


In [134]:
d.rename(columns={"value": "importance"}, inplace='TRUE')


In [135]:
d.to_csv(root_folder+"sample-notebook/feature_importance.csv", index=False)